In [ ]:
import glob
import pandas as pd
import plotly.graph_objects as go
import re
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras_preprocessing import image
from tensorflow.keras import callbacks
import os 
import shutil
img_height=224
img_width=224

In [ ]:
# traning data handle
data_path='../input/classification-of-plants-of-southeast-asia/bali-26_train/bali-26_train'
data=[]
for folder_path in glob.glob('{}/*'.format(data_path)):
    label=folder_path.split('/')[-1]
    for file_path in glob.glob('{}/*.jpg'.format(folder_path)):
        data.append([file_path,label])
df = pd.DataFrame(columns=['filepath', 'label'], data=data)
df.to_csv('data.csv',index=False)
df.head()

In [ ]:
# draw a histogram of the labels
fig=go.Figure(data=[go.Histogram(x=df['label'],xbins=dict(start=0,end=26,size=1))])
fig.update_layout(title='Histogram of labels')
fig.show()

In [ ]:
max_size= 1400


sample_list=[]
groups=df.groupby('label')
for label in df['label'].unique():                 
    group=groups.get_group(label)
    sample_count=len(group)    
    if sample_count> max_size:
        samples=group.sample(max_size, replace=False, weights=None, random_state=123, axis=0).reset_index(drop=True)
    else:
        samples=group.sample(frac=1.0, replace=False, random_state=123, axis=0).reset_index(drop=True)
    sample_list.append(samples)
df=pd.concat(sample_list, axis=0).reset_index(drop=True)
# draw a histogram of the labels
fig=go.Figure(data=[go.Histogram(x=df['label'],xbins=dict(start=0,end=26,size=1))])
fig.update_layout(title='Histogram of labels')
fig.show()

In [ ]:
working_dir=r'./'
aug_dir=os.path.join(working_dir, 'aug')
if os.path.isdir(aug_dir):
    shutil.rmtree(aug_dir)
os.mkdir(aug_dir)
for label in df['label'].unique():
    dir_path=os.path.join(aug_dir,label)    
    os.mkdir(dir_path)
print(os.listdir(aug_dir))

In [ ]:
target = max_size
gen=ImageDataGenerator(horizontal_flip=True,
                       rotation_range=20,
                       width_shift_range=.2,
                       height_shift_range=.2,
                       zoom_range=.2)
groups=df.groupby('label')
for label in df['label'].unique():  # for every class            
    group=groups.get_group(label)  # a dataframe holding only rows with the specified label 
    sample_count=len(group)   # determine how many samples there are in this class  
    if sample_count< target: # if the class has less than target number of images
        aug_img_count=0
        delta=target-sample_count  # number of augmented images to create
        target_dir=os.path.join(aug_dir, label)  # define where to write the images    
        aug_gen=gen.flow_from_dataframe( group,  x_col='filepath', y_col=None, target_size=(img_height,img_width), class_mode=None, batch_size=1,
                                         shuffle=False, save_to_dir=target_dir, save_prefix='aug-',save_format='jpg')
        while aug_img_count<delta:
            images=next(aug_gen)            
            aug_img_count += len(images) 

In [ ]:
aug_fpaths=[]
aug_labels=[]
classlist=os.listdir(aug_dir)
for klass in classlist:
    classpath=os.path.join(aug_dir, klass)     
    flist=os.listdir(classpath)    
    for f in flist:        
        fpath=os.path.join(classpath,f)         
        aug_fpaths.append(fpath)
        aug_labels.append(klass)
Fseries=pd.Series(aug_fpaths, name='filepath')
Lseries=pd.Series(aug_labels, name='label')
aug_df=pd.concat([Fseries, Lseries], axis=1)
print ('length of aug_df" ', len(aug_df))
balance_df=pd.concat([df,aug_df], axis=0).reset_index(drop=True)
balance_df=balance_df.sample(frac=1.0, replace=False, random_state=123, axis=0).reset_index(drop=True)
print ('length of df is: ', len(df)) 

# draw a histogram of the labels
fig=go.Figure(data=[go.Histogram(x=balance_df['label'],xbins=dict(start=0,end=26,size=1))])
fig.update_layout(title='Histogram of labels')
fig.show()

In [ ]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):    
    print("Name:", gpu.name, "  Type:", gpu.device_type)
    tf.compat.v2.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# ImageDataGenerator
train_data_dir='../input/classification-of-plants-of-southeast-asia/bali-26_train/bali-26_train'
batch_size=64

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.05 # set validation split
                                  ) 
# preview the train_datagen image
pic=image.load_img('../input/classification-of-plants-of-southeast-asia/bali-26_train/bali-26_train/durian/10022.jpg')
pic=pic.resize((img_height,img_width))
pic_arr=image.img_to_array(pic)
pic_arr=pic_arr.reshape((1,)+pic_arr.shape)
num_img=4
fig=plt.figure(figsize=(num_img,num_img))
fig.set_size_inches(18.5, 10.5)
count=0
for b in train_datagen.flow(pic_arr):
    plt.subplot(num_img,num_img,count+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(b[0])
    count+=1
    if count==num_img*num_img:
        break
plt.show()



In [ ]:
# load data
train_generator = train_datagen.flow_from_dataframe(
    balance_df,
    x_col='filepath', y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size, class_mode='categorical',
    shuffle=True,
    subset='training')  # set as training data

validation_generator = train_datagen.flow_from_dataframe(
    balance_df,
    x_col='filepath', y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size, class_mode='categorical',
    subset='validation')  # set as validation data


labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [ ]:
print(train_generator.samples,len(train_generator))
print(train_generator.samples//batch_size)
print(type(train_generator))

In [ ]:
# xem thu data
img, label = train_generator.next()
print(img.shape)
print(label.shape)
plt.imshow(img[0])
print(labels[np.argmax(label[0])])
print(np.min(img),np.max(img))

In [ ]:
# preview image
fig=plt.figure(figsize=(5,5))
fig.set_size_inches(18.5, 10.5)

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img[i%batch_size])
    plt.xlabel(labels[np.argmax(label[i%batch_size])])
    if (i%batch_size==0):
        img, label = train_generator.next()
plt.show()

In [ ]:
from tensorflow.keras.applications  import InceptionResNetV2
from tensorflow.keras.models import Model
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout

In [ ]:
    resnet = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3),pooling='avg')
    output = resnet.layers[-1].output
    output = tf.keras.layers.Flatten()(output)
    resnet = Model(resnet.input, output)
    set_trainable = False
    res_name = []
    for layer in resnet.layers:
        res_name.append(layer.name)
    for layer in resnet.layers:
        if layer.name in res_name[-447:]:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

In [ ]:
resnet.summary()

In [ ]:
def createModel():
    model = models.Sequential()
    model.add(resnet)
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    # 26 category
    model.add(Dense(26,activation='softmax'))
    model.summary()
    model.compile(optimizer ="adam",                
                  steps_per_execution = 50,
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model


In [ ]:
model=createModel()

In [ ]:
## let train =))
epochs=200
history = model.fit(train_generator,
                          epochs=epochs,
                          validation_data=validation_generator,
                     callbacks=[
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,verbose=1,restore_best_weights=True),
                        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.1, min_lr=0.00001)
                    ]
                        )


In [ ]:
model.save('model.h5')

In [ ]:
try:
    # summarize history 
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    fig=plt.figure(figsize=(8, 8))
    fig.set_size_inches(18.5, 10.5)
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
except:
    pass

In [ ]:
# predict the test for submission
folder_path='../input/classification-of-plants-of-southeast-asia/bali-26_test/bali-26_test'
images= glob.glob('{}/*.jpg'.format(folder_path))
submissions=[]

for image_file in images:
    img = image.load_img(image_file, target_size=(img_height, img_width))
    x = image.img_to_array(img)
    x = x / 255.0
    x = x.reshape((1,)+x.shape)
    predict=model.predict(x)[0]
    
    idd=image_file.split('/')[-1]
    category=labels[np.argmax(predict)]
    submissions.append([idd,category])
df = pd.DataFrame(columns=['id', 'category'], data=submissions)
df.to_csv('submission.csv',index=False)
plt.imshow(img)
print(predict)
print(category)
